In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=82b1e3387b10d6fac8c2986829c5f0cccfa1888d3706d09384f8956e62a1e718
  Stored in directory: /home/jovyan/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
import wget

In [11]:
# import necessary libraries
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
# create sparksession
spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [16]:
url = 'https://data.vermont.gov/resource/786x-sbp3.csv'
filename = wget.download(url)
filename

'786x-sbp3.csv'

In [23]:
# %%timeit
df_pandas = pd.read_csv(filename)
df_pandas.head(5)

,qtrending,govtunit,unitno,vno,vendor,city,state,description,objcode,amt,account,acctno,fund_description,fund
0,2019-12-31T00:00:00.000,Human Resources-Gov'tal,1120,43671,Agency of Digital Services-CIT-Cust Supp,Montpelier,VT,DHR - VTHR Operations,1120080000,23.46,Telecom-Wireless Phone Service,516659,Financial Management Fund,59300
1,2019-12-31T00:00:00.000,Tax,1140,40668,Vernon Town Clerk,Vernon,VT,Tax Operation Costs,1140010000,15.00,Recording & Other Fees,507620,General Fund,10000
2,2019-12-31T00:00:00.000,Agency of Digital Services,1105,43658,Buildings & General Services-Print Shop,Montpelier,VT,ADS Chief Financial Office,1105500501,10.00,Printing & Binding-BGS Copy Ct,517005,Information Technology,58100
3,2019-12-31T00:00:00.000,Human Resources-Gov'tal,1120,45300,Buildings & General Svcs-Purchasing Card,Montpelier,VT,PERSONNEL OPERATIONS,1120010000,153.72,Telecom-Conf Calling Services,516658,General Fund,10000
4,2019-12-31T00:00:00.000,Libraries,1130,42646,Fair Haven Free Library,Fair Haven,VT,Library & Information Services,1130033000,390.00,Grants,550220,Federal Revenue Fund,22005


In [24]:
df_pandas.describe

<bound method NDFrame.describe of                    qtrending                    govtunit  unitno    vno  \
0    2019-12-31T00:00:00.000     Human Resources-Gov'tal    1120  43671   
1    2019-12-31T00:00:00.000                         Tax    1140  40668   
2    2019-12-31T00:00:00.000  Agency of Digital Services    1105  43658   
3    2019-12-31T00:00:00.000     Human Resources-Gov'tal    1120  45300   
4    2019-12-31T00:00:00.000                   Libraries    1130  42646   
..                       ...                         ...     ...    ...   
995  2019-12-31T00:00:00.000                         Tax    1140  40241   
996  2019-12-31T00:00:00.000                         Tax    1140  40734   
997  2019-12-31T00:00:00.000                         Tax    1140  40653   
998  2019-12-31T00:00:00.000                         Tax    1140  40590   
999  2019-12-31T00:00:00.000                         Tax    1140  40357   

                                       vendor           city stat

In [25]:
df = spark.read.csv(filename, header='true', inferSchema = True)
df

DataFrame[qtrending: timestamp, govtunit: string, unitno: int, vno: int, vendor: string, city: string, state: string, description: string, objcode: int, amt: double, account: string, acctno: int, fund_description: string, fund: int]

In [28]:
df = df.withColumn("Amount", df["amt"].cast("double"))
df, df.columns

(DataFrame[qtrending: timestamp, govtunit: string, unitno: int, vno: int, vendor: string, city: string, state: string, description: string, objcode: int, amt: double, account: string, acctno: int, fund_description: string, fund: int, Amount: double],
 ['qtrending',
  'govtunit',
  'unitno',
  'vno',
  'vendor',
  'city',
  'state',
  'description',
  'objcode',
  'amt',
  'account',
  'acctno',
  'fund_description',
  'fund',
  'Amount'])

In [34]:
df.show(5)

+-------------------+--------------------+------+-----+--------------------+----------+-----+--------------------+----------+------+--------------------+------+--------------------+-----+------+
|          qtrending|            govtunit|unitno|  vno|              vendor|      city|state|         description|   objcode|   amt|             account|acctno|    fund_description| fund|Amount|
+-------------------+--------------------+------+-----+--------------------+----------+-----+--------------------+----------+------+--------------------+------+--------------------+-----+------+
|2019-12-31 00:00:00|Human Resources-G...|  1120|43671|Agency of Digital...|Montpelier|   VT|DHR - VTHR Operat...|1120080000| 23.46|Telecom-Wireless ...|516659|Financial Managem...|59300| 23.46|
|2019-12-31 00:00:00|                 Tax|  1140|40668|   Vernon Town Clerk|    Vernon|   VT| Tax Operation Costs|1140010000|  15.0|Recording & Other...|507620|        General Fund|10000|  15.0|
|2019-12-31 00:00:00|Agen

In [35]:
df.describe().show()

+-------+--------------------+------------------+------------------+----------------+---------+-----+--------------------+--------------------+------------------+--------------------+------------------+--------------------+-----------------+------------------+
|summary|            govtunit|            unitno|               vno|          vendor|     city|state|         description|             objcode|               amt|             account|            acctno|    fund_description|             fund|            Amount|
+-------+--------------------+------------------+------------------+----------------+---------+-----+--------------------+--------------------+------------------+--------------------+------------------+--------------------+-----------------+------------------+
|  count|                1000|              1000|              1000|            1000|     1000| 1000|                1000|                1000|              1000|                1000|              1000|               

In [37]:
df.createOrReplaceTempView('VermontVendor')
spark.sql(
'''
SELECT qtrending, govtunit, Amount, State FROM VermontVendor
LIMIT 10
'''
).show()

+-------------------+--------------------+-------+-----+
|          qtrending|            govtunit| Amount|State|
+-------------------+--------------------+-------+-----+
|2019-12-31 00:00:00|Human Resources-G...|  23.46|   VT|
|2019-12-31 00:00:00|                 Tax|   15.0|   VT|
|2019-12-31 00:00:00|Agency of Digital...|   10.0|   VT|
|2019-12-31 00:00:00|Human Resources-G...| 153.72|   VT|
|2019-12-31 00:00:00|           Libraries|  390.0|   VT|
|2019-12-31 00:00:00|                 Tax|   15.0|   VT|
|2019-12-31 00:00:00|Agency of Digital...|18750.0|   IL|
|2019-12-31 00:00:00|Agency of Digital...|   50.0|   VT|
|2019-12-31 00:00:00|Agency of Digital...| 8248.0|   IL|
|2019-12-31 00:00:00|Human Resources-G...|  27.11|   VT|
+-------------------+--------------------+-------+-----+



In [39]:
df.select('qtrending', 'govtunit', 'Amount', 'State').show()

+-------------------+--------------------+--------+-----+
|          qtrending|            govtunit|  Amount|State|
+-------------------+--------------------+--------+-----+
|2019-12-31 00:00:00|Human Resources-G...|   23.46|   VT|
|2019-12-31 00:00:00|                 Tax|    15.0|   VT|
|2019-12-31 00:00:00|Agency of Digital...|    10.0|   VT|
|2019-12-31 00:00:00|Human Resources-G...|  153.72|   VT|
|2019-12-31 00:00:00|           Libraries|   390.0|   VT|
|2019-12-31 00:00:00|                 Tax|    15.0|   VT|
|2019-12-31 00:00:00|Agency of Digital...| 18750.0|   IL|
|2019-12-31 00:00:00|Agency of Digital...|    50.0|   VT|
|2019-12-31 00:00:00|Agency of Digital...|  8248.0|   IL|
|2019-12-31 00:00:00|Human Resources-G...|   27.11|   VT|
|2019-12-31 00:00:00|           Libraries|51028.75|   CA|
|2019-12-31 00:00:00|           Libraries|   390.0|   VT|
|2019-12-31 00:00:00|                 Tax|    15.0|   VT|
|2019-12-31 00:00:00|                 Tax|    45.0|   VT|
|2019-12-31 00